**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8** 

In [89]:
!pip install apyori --break

Defaulting to user installation because normal site-packages is not writeable


In [90]:
import pandas as pd
from apyori import apriori

In [91]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('../bases/MercadoSim.csv', sep=';',header=None)
base

,0,1,2,3,4,5,6
0,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,Pao,Manteiga,NaN,NaN
2,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
3,Leite,Cafe,NaN,Pao,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,Pao,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Feijao
8,NaN,NaN,NaN,NaN,NaN,Arroz,Feijao
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [92]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [93]:
transacoes = [] 
for i in range(len(base)):
    transacoes.append([ str(base.values[i, j]) for j in range(base.shape[1]) if not pd.isna(base.values[i, j])])

In [94]:
transacoes

[['Cafe', 'Pao', 'Manteiga'],
 ['Leite', 'Cerveja', 'Pao', 'Manteiga'],
 ['Cafe', 'Pao', 'Manteiga'],
 ['Leite', 'Cafe', 'Pao', 'Manteiga'],
 ['Cerveja'],
 ['Manteiga'],
 ['Pao'],
 ['Feijao'],
 ['Arroz', 'Feijao'],
 ['Arroz']]

In [95]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [96]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.8)
saida = list(regras) 
print(len(saida))
print(saida)

4
[RelationRecord(items=frozenset({'Manteiga', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)]), RelationRecord(items=frozenset({'Cafe', 'Pao'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Pao'}), confidence=1.0, lift=2.0)]), RelationRecord(items=frozenset({'Manteiga', 'Pao'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Pao'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Pao'}), items_add=frozenset({'Manteiga'}), confidence=0.8, lift=1.6)]), RelationRecord(items=frozenset({'Manteiga', 'Cafe', 'Pao'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Manteiga', 'Pao'}), confidence=1.0, lift=2.5), OrderedStatistic(items_base=frozenset({'Manteiga', 'Cafe'}), items_add=frozenset({'Pao'

In [97]:
print(saida[0])

RelationRecord(items=frozenset({'Manteiga', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)])


**Para facilitar a visualização, veja como transformar isso em um dataframe** 

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [98]:
itemsets_com_suporte = []
for resultado in saida:
    itemset = list(resultado[0]) 
    suporte = resultado[1]        
    itemsets_com_suporte.append((itemset, suporte))

df_itemsets = pd.DataFrame(itemsets_com_suporte, columns=['Itemset', 'Suporte'])

#df_itemsets = df_itemsets.sort_values(by='Suporte', ascending=False)

print("Itemsets gerados com seus respectivos suportes:")
print(df_itemsets)

Itemsets gerados com seus respectivos suportes:
                 Itemset  Suporte
0       [Manteiga, Cafe]      0.3
1            [Cafe, Pao]      0.3
2        [Manteiga, Pao]      0.4
3  [Manteiga, Cafe, Pao]      0.3


In [99]:
df_itemsets

,Itemset,Suporte
0,"[Manteiga, Cafe]",0.3
1,"[Cafe, Pao]",0.3
2,"[Manteiga, Pao]",0.4
3,"[Manteiga, Cafe, Pao]",0.3


In [100]:
lhs = []
rhs = []
suporte = []
confianca = []
lift = []

for resultado in saida:
  #print(resultado)
  s = resultado[1]
  #print(s)
  result_rules = resultado[2]
  #print(result_rule)
  for result_rule in result_rules:
    #print(result_rule)
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    #print(a, ' - ', b, ' - ', c, ' - ', l)
    lhs.append(a)
    rhs.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Lhs': lhs, 'Rhs': rhs, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [101]:
RegrasFinais

,Lhs,Rhs,suporte,confianca,lift
0,[Cafe],[Manteiga],0.3,1.0,2.0
1,[Cafe],[Pao],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
4,[Cafe],"[Manteiga, Pao]",0.3,1.0,2.5
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
6,"[Cafe, Pao]",[Manteiga],0.3,1.0,2.0


**Podemos ordenar estas regras por uma métrica desejada**

In [102]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Lhs,Rhs,suporte,confianca,lift
4,[Cafe],"[Manteiga, Pao]",0.3,1.0,2.5
0,[Cafe],[Manteiga],0.3,1.0,2.0
1,[Cafe],[Pao],0.3,1.0,2.0
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
6,"[Cafe, Pao]",[Manteiga],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
